# Data Cleaning

In [14]:
import pandas as pd
import numpy as np

df = pd.read_csv("../data/raw/home_price.csv")
df.head()

,Net_Metrekare,Brüt_Metrekare,Oda_Sayısı,Bulunduğu_Kat,Eşya_Durumu,Binanın_Yaşı,Isıtma_Tipi,Fiyat,Şehir,Binanın_Kat_Sayısı,Kullanım_Durumu,Yatırıma_Uygunluk,Takas,Tapu_Durumu,Banyo_Sayısı
0,120,150.0,4.0,4.Kat,Eşyalı,21 Ve Üzeri,Kombi Doğalgaz,950000.0,adana,6,Boş,NaN,Var,Kat Mülkiyeti,1.0
1,100,125.0,4.0,3.Kat,Boş,4,Kombi Doğalgaz,1250000.0,adana,10,Boş,NaN,Var,NaN,1.0
2,89,95.0,3.0,4.Kat,Boş,0 (Yeni),Kombi Doğalgaz,1750000.0,adana,14,Boş,Uygun,Var,NaN,1.0
3,40,55.0,2.0,6.Kat,Boş,0 (Yeni),Kombi Doğalgaz,1300000.0,adana,14,Boş,Uygun,Yok,NaN,1.0
4,140,150.0,4.0,Düz Giriş (Zemin),Boş,5-10,Klimalı,1700000.0,adana,4,Boş,Uygun,Yok,Kat Mülkiyeti,1.0


In [19]:
for col in df : 
    print(df[col])

0        120
1        100
2         89
3         40
4        140
        ... 
20321    130
20322     95
20323    125
20324     50
20325    110
Name: Net_Metrekare, Length: 20109, dtype: int64
0        150
1        125
2         95
3         55
4        150
        ... 
20321    140
20322    115
20323    155
20324     70
20325    130
Name: Brüt_Metrekare, Length: 20109, dtype: int64
0        4
1        4
2        3
3        2
4        4
        ..
20321    4
20322    3
20323    4
20324    2
20325    3
Name: Oda_Sayısı, Length: 20109, dtype: int64
0                    4.Kat
1                    3.Kat
2                    4.Kat
3                    6.Kat
4        Düz Giriş (Zemin)
               ...        
20321                4.Kat
20322                2.Kat
20323                1.Kat
20324         Yüksek Giriş
20325                2.Kat
Name: Bulunduğu_Kat, Length: 20109, dtype: object
0        Eşyalı
1           Boş
2           Boş
3           Boş
4           Boş
          ...  
20321

## Outliers and Duplicated Rows Handling 

In [15]:
import matplotlib.pyplot as plt

numerical_columns = ["Net_Metrekare", "Brüt_Metrekare", "Oda_Sayısı", "Fiyat", "Binanın_Kat_Sayısı", "Banyo_Sayısı"]
categorial_columns = ["Bulunduğu_Kat","Eşya_Durumu","Binanın_Yaşı","Isıtma_Tipi","Şehir","Kullanım_Durumu","Yatırıma_Uygunluk","Takas"]

# Here I deleted any duplicated rows    
df.drop_duplicates(keep='first', inplace=True)

df = df[df["Net_Metrekare"] < 10000]
df = df[df["Brüt_Metrekare"] < 10000]
df = df[df["Fiyat"] < 7000000000]

# for col in numerical_columns:
    
#     counter = 0
# Here I created a boxplot to visualize the outliers.
    # df.boxplot(column = col)
    
    # q1,q3 = df[col].quantile([0.25,0.75])
    # iqr = q3 - q1
    # limit_1 = q1-1.5*iqr
    # limit_2 = q3+1.5*iqr
    # df[col] = df[col].clip(upper = df[col].quantile(0.97))
    # df[col] = df[col].clip(lower = df[col].quantile(0.03)) 


## Imputation

In [16]:
from sklearn.preprocessing import MinMaxScaler , OrdinalEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

# I deleted the Tapu Durumu column here because it is half empty
df = df.drop("Tapu_Durumu" , axis = 1)

# I kept the encoder out of the pipeline because we have categorial, and numerical data. This might cause some data leakage but it is not very significant.
oe = OrdinalEncoder(encoded_missing_value=np.nan)

df[categorial_columns] = oe.fit_transform(df[categorial_columns])

df_train, df_test = train_test_split(df, test_size = 0.2, random_state= 44)

preprocessing_steps = [

    ('scaler', MinMaxScaler()),
    ('imputer', IterativeImputer(estimator=BayesianRidge(), max_iter=10, random_state=0))
]

preprocessing_pipeline = Pipeline(preprocessing_steps)
preprocessing_pipeline.set_output(transform='pandas')

preprocessing_pipeline.fit(df_train)


df = preprocessing_pipeline.transform(df)

minmax_pipeline = preprocessing_pipeline.named_steps['scaler']

df_array = minmax_pipeline.inverse_transform(df)

df = pd.DataFrame(data=df_array, columns=df.columns, index=df.index).round().astype(int)
df[categorial_columns] = df[categorial_columns]
df[categorial_columns] = oe.inverse_transform(df[categorial_columns])

df = df.dropna()

In [17]:
df.head()

,Net_Metrekare,Brüt_Metrekare,Oda_Sayısı,Bulunduğu_Kat,Eşya_Durumu,Binanın_Yaşı,Isıtma_Tipi,Fiyat,Şehir,Binanın_Kat_Sayısı,Kullanım_Durumu,Yatırıma_Uygunluk,Takas,Banyo_Sayısı
0,120,150,4,4.Kat,Eşyalı,21 Ve Üzeri,Kombi Doğalgaz,950000,adana,6,Boş,Uygun,Var,1
1,100,125,4,3.Kat,Boş,4,Kombi Doğalgaz,1250000,adana,10,Boş,Uygun,Var,1
2,89,95,3,4.Kat,Boş,0 (Yeni),Kombi Doğalgaz,1750000,adana,14,Boş,Uygun,Var,1
3,40,55,2,6.Kat,Boş,0 (Yeni),Kombi Doğalgaz,1300000,adana,14,Boş,Uygun,Yok,1
4,140,150,4,Düz Giriş (Zemin),Boş,5-10,Klimalı,1700000,adana,4,Boş,Uygun,Yok,1


### Saving the dataset 

In [20]:
df.to_csv("../data/processed/cleaned_data.csv" , index=False )